In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs found. Training will default to CPU.")

2024-04-23 13:23:47.739099: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 13:23:48.176290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 13:23:48.176359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 13:23:48.275234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 13:23:48.606970: I tensorflow/core/platform/cpu_feature_guar

No GPUs found. Training will default to CPU.


In [2]:
data_dir = 'data'

In [3]:
# Define parameters
img_width, img_height = 128, 128
batch_size = 32

In [4]:
# Create an ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1765  # Approx 15% for validation, when combined with 70% training
)


In [5]:
# Create an ImageDataGenerator for testing (no augmentation, just scaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Prepare iterators
train_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Set as training data
)


Found 9783 images belonging to 2 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Set as validation data
)


Found 2096 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
    data_dir + '/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.


In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 28800)             0         
                                                                 
 dense (Dense)               (None, 64)                1

In [10]:
# Run this cell to train model

import tensorflow as tf
import keras
import scipy

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)
print("Scipy Version:", scipy.__version__)

epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)

TensorFlow Version: 2.15.0
Keras Version: 2.15.0
Scipy Version: 1.12.0
Epoch 1/10
305/305 [==============================] - 119s 387ms/step - loss: 0.5182 - accuracy: 0.7456 - val_loss: 0.5104 - val_accuracy: 0.7447
Epoch 2/10
305/305 [==============================] - 102s 333ms/step - loss: 0.4213 - accuracy: 0.8145 - val_loss: 0.5452 - val_accuracy: 0.7500
Epoch 3/10
305/305 [==============================] - 100s 328ms/step - loss: 0.4007 - accuracy: 0.8284 - val_loss: 0.5241 - val_accuracy: 0.7611
Epoch 4/10
305/305 [==============================] - 119s 391ms/step - loss: 0.3838 - accuracy: 0.8350 - val_loss: 0.5304 - val_accuracy: 0.7716
Epoch 5/10
305/305 [==============================] - 115s 377ms/step - loss: 0.3716 - accuracy: 0.8420 - val_loss: 0.5211 - val_accuracy: 0.7885
Epoch 6/10
305/305 [==============================] - 104s 342ms/step - loss: 0.3729 - accuracy: 0.8418 - val_loss: 0.4645 - val_accuracy: 0.7817
Epoch 7/10
305/305 [==============================] -

In [12]:
# RUN THIS CELL TO LOAD PRETRAINED MODEL

model = keras.models.load_model('model1.h5')

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')

63/63 [==============================] - 5s 65ms/step - loss: 0.3632 - accuracy: 0.8670
Test accuracy: 0.8669999837875366
